<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/quantumfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="blue">**Quantum Finance**

In [ ]:
!pip install cirq -q

In [ ]:
import cirq
import random
import matplotlib.pyplot as plt
import numpy as np

Quantum computing for finance: Overview and prospects: https://www.sciencedirect.com/science/article/pii/S2405428318300571

Quantum computing for finance: https://arxiv.org/abs/2307.11230